In [1]:
# Bank Customer Artificial Neural Network

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [4]:
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

### Data Preprocessing

In [5]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
transformer = ColumnTransformer(
    transformers=[
        ("OneHot",        # Just a name
         OneHotEncoder(), # The transformer class
         [1]              # The column(s) to be applied on.
         )
    ],
    remainder='passthrough' # donot apply anything to the remaining columns
)
X = transformer.fit_transform(X.tolist())
print(X[1])
X = X[:, 1:]

[0.0 0.0 1.0 608 0 41 1 83807.86 1 0 1 112542.58]


#### Splitting the dataset into the Training set and Test set

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

### Feature Scaling

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
print(X_test.shape)
print(X_train.shape)


(2000, 11)
(8000, 11)


In [8]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout


In [9]:
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer (with dropout)
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11)) # 6 nodes in hidden layer, initialize weights uniformly, use rectifier funtion for hidden layer, except 11 input nodes
classifier.add(Dropout(0.1)) # Disable 10% of the neurons on each iteration

# Adding the second hidden layer (with dropout)
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu')) # input_dim already specified in previous hidden layer
classifier.add(Dropout(0.1))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid')) # only 1 node in output layer, use sigmoid function for probability

In [10]:
# Compiling the ANN - apply stochastic gradient descent
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy']) # loss defined this way since output is binary, only 1 output node

# Fitting the ANN to the Training set
history=classifier.fit(X_train, y_train, batch_size = 10, epochs = 30)  #epochs = 10

Epoch 1/30
800/800 [==============================] - 3s 2ms/step - loss: 0.4830 - accuracy: 0.7970
Epoch 2/30
800/800 [==============================] - 1s 2ms/step - loss: 0.4329 - accuracy: 0.7974
Epoch 3/30
800/800 [==============================] - 1s 2ms/step - loss: 0.4282 - accuracy: 0.7974
Epoch 4/30
800/800 [==============================] - 1s 2ms/step - loss: 0.4236 - accuracy: 0.8011
Epoch 5/30
800/800 [==============================] - 1s 2ms/step - loss: 0.4257 - accuracy: 0.8240
Epoch 6/30
800/800 [==============================] - 1s 2ms/step - loss: 0.4223 - accuracy: 0.8273
Epoch 7/30
800/800 [==============================] - 1s 2ms/step - loss: 0.4247 - accuracy: 0.8299
Epoch 8/30
800/800 [==============================] - 1s 2ms/step - loss: 0.4228 - accuracy: 0.8294
Epoch 9/30
800/800 [==============================] - 1s 2ms/step - loss: 0.4246 - accuracy: 0.8303
Epoch 10/30
800/800 [==============================] - 1s 2ms/step - loss: 0.4210 - accuracy: 0.8320

In [11]:
## Dump the model 

from joblib import dump, load

dump(classifier, 'model.joblib')

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\dense_2
......vars
.........0
.........1
...layers\dropout
......vars
...layers\dropout_1
......vars
...metrics\mean
......vars
.........0
.........1
...metrics\mean_metric_wrapper
......vars
.........0
.........1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2022-12-19 12:00:47         2213
metadata.json                                  2022-12-19 12:00:47           64
variables.h5                                   2022-12-19 12:00:47        27112


['model.joblib']

In [ ]:


fig_w, fig_h = (4.5, 3.5)
plt.rcParams.update({'figure.figsize': (fig_w, fig_h)})
fig, axs = plt.subplots(1, 2, figsize=(2*fig_w,fig_h))

plt.sca(axs[0])
plt.plot(history.history['accuracy'])

plt.ylabel('Accuracy')
plt.title('Categorical Accuracy')

plt.sca(axs[1])
plt.plot(history.history['loss'])
plt.ylabel('Loss')
plt.title('Loss')

plt.tight_layout()
fig.text(0.5, 0.01,'Epochs', ha='center')
plt.show()

In [ ]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5) # Set threshold for returning true or false

# EXAMPLE: Predicting a single new observation
'''Predict if the customer with the following informations will leave the bank: 
Geography: France
Credit Score: 600
Gender: Male
Age: 40 years old
Tenure: 3 years
Balance: $60000
Number of Products: 2
Does this customer have a credit card ? Yes
Is this customer an Active Member: Yes
Estimated Salary: $50000
'''
new_prediction = classifier.predict(sc.transform(np.array([[0.0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]]))) # Manually encode all independent variables, scale to training set
new_prediction = (new_prediction > 0.5)
print(new_prediction)
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)